# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [14]:
# Memuat dataset
data = pd.read_csv("https://raw.githubusercontent.com/faarismuda/Dicoding-ML-Shop-Customers/main/Dataset/clustering_results.csv")

# Konversi kategorial ke numerik
data['gender'] = data['gender'].map({'Male': 0, 'Female': 1})

mapping_freq = {'rare': 0, 'occasional': 1, 'frequent': 2}
data['purchase_frequency'] = data['purchase_frequency'].map(mapping_freq)

mapping_loyalty = {'Regular': 0, 'Silver': 1, 'Gold': 2}
data['loyalty_status'] = data['loyalty_status'].map(mapping_loyalty)

# Menampilkan 5 baris pertama
data.head()

,age,gender,income,education,region,loyalty_status,purchase_frequency,purchase_amount,product_category,promotion_usage,satisfaction_score,Cluster
0,27,0,40682,Bachelor,East,2,2,18249,Books,0,6,1
1,29,0,15317,Masters,West,0,0,4557,Clothing,1,6,0
2,37,0,38849,Bachelor,West,1,0,11822,Clothing,0,6,1
3,30,0,11568,HighSchool,South,0,2,4098,Food,0,7,0
4,31,1,46952,College,North,0,1,19685,Clothing,1,5,1


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [15]:
# Memisahkan fitur dan label dengan menghapus fitur yang tidak relevan
X = data.drop(['Cluster', 'product_category', 'region', 'education'], axis=1)
y = data['Cluster']

# Split data menjadi training & testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print("Ukuran X_train:", X_train.shape)
print("Ukuran y_train:", y_train.shape)
print("Ukuran X_test:", X_test.shape)
print("Ukuran y_test:", y_test.shape)

Ukuran X_train: (80000, 8)
Ukuran y_train: (80000,)
Ukuran X_test: (20000, 8)
Ukuran y_test: (20000,)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [9]:
# Daftar model yang akan diuji
models = {
    "Support Vector Machine (SVM)": SVC(kernel='rbf', C=1.0, gamma='scale'),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

# Dictionary untuk menyimpan hasil evaluasi
results = {}

# K-Fold Cross Validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Loop untuk melatih setiap model
for name, model in models.items():
    print(f"\nTraining {name}...")

    # Pipeline model
    pipeline = Pipeline([
        ('classifier', model)
    ])

    # Cross Validation
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=kf, scoring='accuracy')

    # Training model
    pipeline.fit(X_train, y_train)

    # Prediksi pada test set
    y_pred = pipeline.predict(X_test)

    # Simpan hasil evaluasi
    results[name] = {
        "model": pipeline,
        "cv_scores": cv_scores,
        "y_pred": y_pred
    }


Training Support Vector Machine (SVM)...

Training Logistic Regression...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c


Training Random Forest...


Tulis narasi atau penjelasan algoritma yang Anda gunakan.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [10]:
print("\n=== EVALUASI MODEL ===")

for name, result in results.items():
    print(f"\n=== {name} ===")

    # Cross Validation Metrics
    print(f"Cross-Validation Accuracy Scores: {result['cv_scores']}")
    print(f"Mean Cross-Validation Accuracy: {np.mean(result['cv_scores'])}")
    print(f"Standard Deviation: {np.std(result['cv_scores'])}")
    print(f"Variance: {np.var(result['cv_scores'])}")

    # Evaluasi pada test set
    y_pred = result['y_pred']
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"\nFinal Evaluation Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")

    print(f"\nClassification Report:\n{classification_report(y_test, y_pred)}")
    print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}")


=== EVALUASI MODEL ===

=== Support Vector Machine (SVM) ===
Cross-Validation Accuracy Scores: [0.999875 0.99975  0.99975  0.999875 1.       0.999875 0.999875 0.999625
 1.       0.999375]
Mean Cross-Validation Accuracy: 0.9998000000000001
Standard Deviation: 0.0001785357107135609
Variance: 3.1874999999996307e-08

Final Evaluation Accuracy: 0.9999
Precision: 0.9999
Recall: 0.9999
F1-score: 0.9999

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9965
           1       1.00      1.00      1.00     10035

    accuracy                           1.00     20000
   macro avg       1.00      1.00      1.00     20000
weighted avg       1.00      1.00      1.00     20000

Confusion Matrix:
[[ 9964     1]
 [    1 10034]]

=== Logistic Regression ===
Cross-Validation Accuracy Scores: [0.9955   0.971125 0.99225  0.97025  0.99025  0.979875 0.978875 0.983125
 0.975    0.994875]
Mean Cross-Validation Accuracy: 0.9831125000

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [11]:
# Hasil klasifikasi sudah memiliki akurasi tinggi

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [12]:
# Hasil klasifikasi sudah memiliki akurasi tinggi

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

1. **Support Vector Machine (SVM)**

Support Vector Machine (SVM) menunjukkan kinerja yang sangat tinggi dengan akurasi sebesar 0.9999. Precision, recall, dan F1-score juga berada pada angka 0.9999, yang mengindikasikan keseimbangan prediksi yang sangat baik untuk kedua kelas. Confusion matrix menunjukkan hanya terdapat 2 kesalahan dari 20.000 sampel, yang menunjukkan tingkat kesalahan yang sangat kecil. Variansi yang rendah (3.1875e-08) menunjukkan bahwa model sangat stabil dalam berbagai lipatan cross-validation. Namun, akurasi yang sangat tinggi ini juga dapat mengindikasikan kemungkinan overfitting, yang berarti model mungkin terlalu menyesuaikan diri dengan data latih dan tidak bekerja optimal untuk data baru. Untuk mengatasi kemungkinan overfitting, disarankan untuk menguji model pada data yang benar-benar baru serta mencoba teknik regulasi seperti menyesuaikan parameter C dalam SVM.

2. **Logistic Regression**

Logistic Regression memiliki akurasi yang cukup baik, yaitu 0.98295, namun sedikit lebih rendah dibandingkan dengan SVM dan Random Forest. Precision, recall, dan F1-score juga berada di sekitar angka 0.9829, yang menunjukkan keseimbangan dalam mengenali kedua kelas. Namun, confusion matrix mengindikasikan adanya 341 kesalahan prediksi, yang lebih banyak dibandingkan dua model lainnya. Selain itu, variansi yang lebih tinggi (8.2963e-05) menunjukkan bahwa model lebih bervariasi dibandingkan SVM. Salah satu masalah yang muncul adalah recall yang lebih rendah dibandingkan model lain, dengan 144 false negatives pada kelas 1. Hal ini bisa menjadi indikasi bahwa model mengalami underfitting, yang berarti model tidak cukup kompleks untuk menangkap pola data dengan baik. Untuk meningkatkan kinerja model ini, disarankan untuk melakukan tuning parameter regulasi (misalnya, dengan menggunakan penalty L1/L2), menambahkan lebih banyak fitur, atau menyesuaikan threshold agar recall lebih optimal.

3. **Random Forest**

Random Forest menunjukkan kinerja yang sangat tinggi dengan akurasi sebesar 0.999, hampir menyamai SVM. Precision, recall, dan F1-score juga berada di angka 0.999, yang menunjukkan keseimbangan yang sangat baik dalam prediksi. Confusion matrix menunjukkan 20 kesalahan prediksi, yang masih sangat kecil dibandingkan dengan jumlah total sampel. Selain itu, variansi yang relatif rendah (1.6328e-07) menunjukkan bahwa model sangat stabil. Namun, akurasi yang sangat tinggi ini juga dapat mengindikasikan kemungkinan overfitting, terutama jika model terlalu kompleks dan tidak bekerja optimal di data baru. Selain itu, efisiensi komputasi menjadi perhatian karena Random Forest membutuhkan lebih banyak sumber daya dibandingkan dengan Logistic Regression. Untuk mengatasi hal ini, disarankan untuk menggunakan teknik pruning atau membatasi jumlah pohon guna mengurangi kompleksitas model, serta melakukan evaluasi dengan dataset baru untuk memastikan tidak ada overfitting.

**Kesimpulan**

Jika prioritas utama adalah mencapai akurasi setinggi mungkin, maka SVM dan Random Forest adalah pilihan terbaik, meskipun perlu diuji lebih lanjut untuk memastikan tidak terjadi overfitting. Jika dibutuhkan model yang lebih sederhana dan cepat, Logistic Regression dapat menjadi alternatif, meskipun performanya lebih rendah dibandingkan model lain. Jika keseimbangan antara akurasi dan interpretabilitas menjadi pertimbangan utama, Random Forest dapat digunakan dengan tuning yang lebih baik untuk menghindari overfitting.